# Training TopKSAE Model

This notebook demonstrates how to:
1. Load a language model and dataset
2. Save activations from a specific layer
3. Train a TopK Sparse Autoencoder (TopKSAE) on those activations using the new `SaeTrainer` composite class
4. Save the trained TopKSAE model

The training uses overcomplete's `train_sae` functions via the `SaeTrainer` composite class, which is automatically available on all SAE instances via `sae.trainer`.

All files (trained TopKSAE model, training metadata, activations) will be saved under `store/{model_id}/` for organized, model-specific storage.


In [53]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
import os
from pathlib import Path
from datetime import datetime

# Silence Hugging Face tokenizers fork/parallelism warnings
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

from amber.adapters import TextDataset
from amber.language_model.language_model import LanguageModel
from amber.mechanistic.sae.modules.topk_sae import TopKSae, TopKSAETrainingConfig
from amber.store import LocalStore

print("✅ Imports completed")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Imports completed


In [54]:
# Configuration
MODEL_ID = "sshleifer/tiny-gpt2"  # Small model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 1000  # Number of text samples to use
MAX_LENGTH = 64  # Maximum sequence length
BATCH_SIZE_SAVE = 16  # Batch size for saving activations
BATCH_SIZE_TRAIN = 32  # Batch size for SAE training

# TopKSAE configuration
TOP_K = 8  # Number of top activations to keep (sparsity parameter)

# Choose which layer to hook - you can inspect available layers with model.layers.print_layer_names()
LAYER_SIGNATURE = 'gpt2lmheadmodel_transformer_h_0_attn_c_attn'  # Attention layer (better activations)

# Storage locations - will be updated after model loading to use model_id
STORE_DIR = Path("store/train_sae")
DATASET_DIR = Path("store/cache")
RUN_ID = f"topk_sae_training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
# Model-specific paths will be set after loading the model
SAE_MODEL_PATH = None  # Will be set to store/{model_id}/topk_sae_model.pt
METADATA_PATH = None  # Will be set to store/{model_id}/training_metadata.json

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else None  # Use half precision on GPU

# Wandb configuration (optional - set use_wandb=True in training config to enable)
WANDB_PROJECT = "amber_playground"  # Wandb project name
WANDB_ENTITY = "amber_team"

print("🚀 Starting TopKSAE Training Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📊 Dataset: {HF_DATASET}")
print(f"🎯 Target layer: {LAYER_SIGNATURE}")
print(f"🔢 TopK parameter: {TOP_K}")
print()

# Create output directories
DATASET_DIR.mkdir(parents=True, exist_ok=True)
STORE_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Output directories created")


🚀 Starting TopKSAE Training Example
📱 Using device: cpu
🔧 Model: sshleifer/tiny-gpt2
📊 Dataset: roneneldan/TinyStories
🎯 Target layer: gpt2lmheadmodel_transformer_h_0_attn_c_attn
🔢 TopK parameter: 8

✅ Output directories created


In [55]:
# Step 1: Load language model with context
print("📥 Loading language model...")

# Load model first to get model_idstore = LocalStore(MODEL_DIR)

store = LocalStore(STORE_DIR)
lm = LanguageModel.from_huggingface(MODEL_ID, store)
lm.model.to(DEVICE)

# Create model-specific directory for organizing all files

# Update paths to use model-specific directory
SAE_MODEL_PATH = STORE_DIR / "topk_sae_model.pt"
METADATA_PATH = STORE_DIR / "training_metadata.json"

# Print available layers for reference
print("🔍 Available layers:")
lm.layers.print_layer_names()
print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"📁 Store base: {STORE_DIR}")
print(f"📁 Store location: {lm.context.store.base_path}")
print(f"💾 SAE model will be saved to: {SAE_MODEL_PATH}")
print(f"💾 Metadata will be saved to: {METADATA_PATH}")


📥 Loading language model...
🔍 Available layers:
gpt2lmheadmodel_transformer: No weight
gpt2lmheadmodel_transformer_wte: torch.Size([50257, 2])
gpt2lmheadmodel_transformer_wpe: torch.Size([1024, 2])
gpt2lmheadmodel_transformer_drop: No weight
gpt2lmheadmodel_transformer_h: No weight
gpt2lmheadmodel_transformer_h_0: No weight
gpt2lmheadmodel_transformer_h_0_ln_1: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_attn: No weight
gpt2lmheadmodel_transformer_h_0_attn_c_attn: torch.Size([2, 6])
gpt2lmheadmodel_transformer_h_0_attn_c_proj: torch.Size([2, 2])
gpt2lmheadmodel_transformer_h_0_attn_attn_dropout: No weight
gpt2lmheadmodel_transformer_h_0_attn_resid_dropout: No weight
gpt2lmheadmodel_transformer_h_0_ln_2: torch.Size([2])
gpt2lmheadmodel_transformer_h_0_mlp: No weight
gpt2lmheadmodel_transformer_h_0_mlp_c_fc: torch.Size([2, 8])
gpt2lmheadmodel_transformer_h_0_mlp_c_proj: torch.Size([8, 2])
gpt2lmheadmodel_transformer_h_0_mlp_act: No weight
gpt2lmheadmodel_transformer_h_0_mlp_dropout: 

In [56]:
# Step 2: Load dataset
print("📥 Loading dataset...")
dataset = TextDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    store=store,
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)
print(f"✅ Loaded {len(dataset)} text samples")


📥 Loading dataset...


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 489931.55 examples/s]

✅ Loaded 1000 text samples


In [57]:
# Step 3: Save activations
print("💾 Saving activations...")

# Use the store that was set on the language model
lm.activations.save_activations_dataset(
    dataset,
    layer_signature=LAYER_SIGNATURE,
    run_name=RUN_ID,
    store=lm.context.store,  # Use the store from language model
    batch_size=BATCH_SIZE_SAVE,
    autocast=False,  # Disable autocast for consistency
)

# Verify activations were saved
batches = lm.context.store.list_run_batches(RUN_ID)
print(f"✅ Saved {len(batches)} batches of activations")
print(f"📁 Run ID: {RUN_ID}")
print(f"📁 Store location: {lm.context.store.base_path}")


💾 Saving activations...


TypeError: save_activations_dataset() got an unexpected keyword argument 'store'

In [49]:
# Step 4: Create TopKSAE model
print("🏗️ Creating TopKSAE model...")

# Get the hidden dimension from the first batch
first_batch = lm.context.store.get_run_batch(RUN_ID, 0)
if isinstance(first_batch, dict):
    activations = first_batch["activations"]
else:
    activations = first_batch[0]  # Assume first tensor is activations

hidden_dim = activations.shape[-1]  # Last dimension is hidden size
print(f"📏 Hidden dimension: {hidden_dim}")

sae = TopKSae(
    n_latents=hidden_dim * 4,
    n_inputs=hidden_dim,
    k=TOP_K,
    device=DEVICE,
)

print(f"🧠 TopKSAE architecture: {hidden_dim} → {sae.context.n_latents} → {hidden_dim}")
print(f"🔢 TopK parameter: {sae.k}")
print(f"🔧 Device: {DEVICE}")


🏗️ Creating TopKSAE model...
📏 Hidden dimension: 6
🧠 TopKSAE architecture: 6 → 24 → 6
🔢 TopK parameter: 8
🔧 Device: cpu


In [ ]:
# Step 5: Train TopKSAE using SaeTrainer
print("🏋️ Training TopKSAE...")
print("📝 Note: Training uses overcomplete's train_sae functions via the SaeTrainer composite class")
print(f"🔧 Trainer available at: sae.trainer (type: {type(sae.trainer).__name__})")
print()

# Configure training parameters
# Note: TopKSAETrainingConfig is an alias for SaeTrainingConfig
# You can also use SaeTrainingConfig directly from sae_trainer module
config = TopKSAETrainingConfig(
    epochs=100,
    batch_size=BATCH_SIZE_TRAIN,
    lr=1e-3,
    l1_lambda=1e-4,  # L1 sparsity penalty
    device=DEVICE,
    dtype=DTYPE,
    max_batches_per_epoch=50,  # Limit batches per epoch for quick training
    verbose=True,  # Enable progress logging
    use_amp=True,
    amp_dtype=DTYPE,
    clip_grad=1.0,  # Gradient clipping (overcomplete parameter)
    monitoring=2,  # Detailed monitoring (0=silent, 1=basic, 2=detailed)
    # Wandb configuration (set use_wandb=True to enable wandb logging)
    use_wandb=True,  # Enable wandb logging
    wandb_entity=WANDB_ENTITY,
    wandb_project=WANDB_PROJECT,
    wandb_name=RUN_ID,  # Use run_id as wandb run name
    wandb_tags=["topk-sae", "sae-training", MODEL_ID.split("/")[-1]],
    wandb_config={
        "model_id": MODEL_ID,
        "dataset": HF_DATASET,
        "layer_signature": LAYER_SIGNATURE,
        "top_k": TOP_K,
        "data_limit": DATA_LIMIT,
        "max_length": MAX_LENGTH,
    },
)

# Train using TopKSAE's train method (which delegates to sae.trainer.train())
# The trainer uses overcomplete's train_sae_amp or train_sae functions internally
history = sae.train(lm.context.store, RUN_ID, LAYER_SIGNATURE, config)

print()
print("✅ Training completed!")
print(f"📈 Final loss: {history['loss'][-1]:.6f}")
print(f"📈 Final reconstruction MSE: {history['recon_mse'][-1]:.6f}")
print(f"📈 Final L1 penalty: {history['l1'][-1]:.6f}")


In [ ]:
# Step 6: Save trained TopKSAE
print("💾 Saving trained TopKSAE...")

# Save using TopKSAE's save method (saves overcomplete model + our metadata)
sae.save(
    name="topk_sae_model",
    path=SAE_MODEL_PATH.parent
)

print(f"✅ TopKSAE saved to: {SAE_MODEL_PATH}")


In [52]:
lm.layers.register_hook(LAYER_SIGNATURE, sae)

'6a51ff1e-9170-4692-8a5a-21fd6e624669'